In [2]:
import pandas as pd
import numpy as np

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
mark_start = "ssss "
mark_end = " eeee"

In [4]:
data_src = []
data_dest = []

In [6]:
for line in open("tur.txt", encoding = "utf-8"):
    en_text, tr_text = line.rstrip().split("\t")
    
    tr_text = mark_start + tr_text + mark_end
    data_src.append(en_text)
    data_dest.append(tr_text)

In [8]:
len(data_src)

946070